# Sentiment Analysis of Twitter Posts
<!-- Notebook name goes here -->
<center><b>Notebook: Data Description, Cleaning, Exploratory Data Analysis, and Preprocessing</b></center>
<br>

**By**: Stephen Borja, Justin Ching, Erin Chua, and Zhean Ganituen.

**Dataset**: Hussein, S. (2021). Twitter Sentiments Dataset [Dataset]. Mendeley. https://doi.org/10.17632/Z9ZW7NT5H2.1

**Motivation**: Every minute, social media users generate a large influx of textual data on live events. Performing sentiment analysis on this data provides a real-time view of public perception, enabling quick insights into the general population’s opinions and reactions.

**Goal**: By the end of the project, our goal is to create and compare supervised learning algorithms for sentiment analysis.

### **Dataset Description**

The Twitter Sentiments Dataset is a dataset that contains nearly 163k tweets from Twitter. The time period of when these were collected is unknown, but it was published to Mendeley Data on May 14, 2021 by Sherif Hussein of Mansoura University.

Tweets were extracted using the Twitter API, but the specifics of how the tweets were selected are unmentioned. The tweets are mostly English with a mix of some Hindi words for code-switching<a name="cite_ref-1"></a>[<sup>[1]</sup>](#cite_note-1). All of them seem to be talking about the political state of India. Most tweets mention Narendra Modi, the current Prime Minister of India.

Each tweet was assigned a label using TextBlob's sentiment analysis (El‑Demerdash, Hussein, & Zaki, 2021), which assigns labels automatically.

What each row and column represents: `each row represents one tweet.`
Number of observations: `162,980`

---
**References**
1. Alaa A. El-Demerdash, J. F. W. Z., Sherif E. Hussein. (2022). Course Evaluation Based on Deep Learning and SSA Hyperparameters Optimization. Computers, Materials & Continua, 71(1), 941–959. doi:10.32604/cmc.2022.021839

---

<a name="cite_note-1"></a>1. [^](#cite_ref-1) Code-switching is the practice of alternating between two languages $L_1$ (the native language) and $L_2$ (the source language) in a conversation. In this context, the code-switching is done to appear more casual since the conversation is done via Twitter (now, X). 

Twitter_Data
- **`clean_text`**: The tweet's text
- **`category`**: The tweet's sentiment category

## 1 project set up
Set up here the imports for the projects (ensure these are installed via uv and is part of the environment). Furthermore, load the dataset here. Also load the raw dataset here.

In [117]:
# imports
# no need to import `torch` here since no training occurs yet.
import numpy as np
import pandas as pd

import unicodedata
import re
import string

from nltk.stem import PorterStemmer

# Load raw data file
df = pd.read_csv("../data/Twitter_Data.csv")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162980 entries, 0 to 162979
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   clean_text  162976 non-null  object 
 1   category    162973 non-null  float64
dtypes: float64(1), object(1)
memory usage: 2.5+ MB


## 2 data cleaning
This section discusses the methodology for data cleaning.

We follow a similar methodology for data cleaning presented in [1].

The cleaning pipeline has four main functions. The first function is the `normalize` function, it normalizes the text input to ASCII-only characters (say, "cómo estás" becomes "como estas") and lowercases alphabetic symbols. The dataset contains Unicode characters (e.g., emojis and accented characters) which the function replaces to the empty string (`''`). Then, `rem_punctuation` removes the punctuation marks and special characters with the empty string. Then, `collapse_whitespace` collapses all whitespace characters to a single space. Formally, it is a transducer from $\Box^+ \mapsto \Box$ where $\Box$ is the whitespace character. Now, since the strings are cleaned at this point, the tokenization step reduces to a mere string split at word boundaries. Finally, with the tokenized string, we can do a final clean by removing the stopwords.

For stop words removal, we refer to the English stopwords dataset defined in NLTK and Wolfram Mathematica [2,3].

---
**References**
1. George, M., & Murugesan, R., Dr. (2024). Improving sentiment analysis of financial news headlines using hybrid Word2Vec-TFIDF feature extraction technique. Procedia Computer Science, 244, 1–8. https://doi.org/10.1016/j.procs.2024.10.172
2. Wolfram Language. (2015). DeleteStopwords. Wolfram Language & System Documentation Center. Retrieved from https://reference.wolfram.com/language/ref/DeleteStopwords.html
3. Bird, S., & Loper, E. (2004, July). NLTK: The Natural Language Toolkit. Proceedings of the ACL Interactive Poster and Demonstration Sessions, 214–217. Retrieved from https://aclanthology.org/P04-3031/

In [106]:
stopwords_set = {
	'0o',
	'0s',
	'3a',
	'3b',
	'3d',
	'6b',
	'6o',
	'a',
	'a1',
	'a2',
	'a3',
	'a4',
	'ab',
	'able',
	'about',
	'above',
	'abst',
	'ac',
	'accordance',
	'according',
	'accordingly',
	'across',
	'act',
	'actually',
	'ad',
	'added',
	'adj',
	'ae',
	'af',
	'affected',
	'affecting',
	'affects',
	'after',
	'afterwards',
	'ag',
	'again',
	'against',
	'ah',
	'ain',
	'aint',
	'aj',
	'al',
	'all',
	'allow',
	'allows',
	'almost',
	'alone',
	'along',
	'already',
	'also',
	'although',
	'always',
	'am',
	'among',
	'amongst',
	'amoungst',
	'amount',
	'an',
	'and',
	'announce',
	'another',
	'any',
	'anybody',
	'anyhow',
	'anymore',
	'anyone',
	'anything',
	'anyway',
	'anyways',
	'anywhere',
	'ao',
	'ap',
	'apart',
	'apparently',
	'appear',
	'appreciate',
	'appropriate',
	'approximately',
	'ar',
	'are',
	'aren',
	'arent',
	'arent',
	'arise',
	'around',
	'as',
	'as',
	'aside',
	'ask',
	'asking',
	'associated',
	'at',
	'au',
	'auth',
	'av',
	'available',
	'aw',
	'away',
	'awfully',
	'ax',
	'ay',
	'az',
	'b',
	'b1',
	'b2',
	'b3',
	'ba',
	'back',
	'bc',
	'bd',
	'be',
	'became',
	'because',
	'become',
	'becomes',
	'becoming',
	'been',
	'before',
	'beforehand',
	'begin',
	'beginning',
	'beginnings',
	'begins',
	'behind',
	'being',
	'believe',
	'below',
	'beside',
	'besides',
	'best',
	'better',
	'between',
	'beyond',
	'bi',
	'bill',
	'biol',
	'bj',
	'bk',
	'bl',
	'bn',
	'both',
	'bottom',
	'bp',
	'br',
	'brief',
	'briefly',
	'bs',
	'bt',
	'bu',
	'but',
	'bx',
	'by',
	'c',
	'c1',
	'c2',
	'c3',
	'ca',
	'call',
	'came',
	'can',
	'cannot',
	'cant',
	'cant',
	'cause',
	'causes',
	'cc',
	'cd',
	'ce',
	'certain',
	'certainly',
	'cf',
	'cg',
	'ch',
	'changes',
	'ci',
	'cit',
	'cj',
	'cl',
	'clearly',
	'cm',
	'cmon',
	'cn',
	'co',
	'com',
	'come',
	'comes',
	'con',
	'concerning',
	'consequently',
	'consider',
	'considering',
	'contain',
	'containing',
	'contains',
	'corresponding',
	'could',
	'couldn',
	'couldnt',
	'couldnt',
	'course',
	'cp',
	'cq',
	'cr',
	'cry',
	'cs',
	'cs',
	'ct',
	'cu',
	'currently',
	'cv',
	'cx',
	'cy',
	'cz',
	'd',
	'd2',
	'da',
	'date',
	'dc',
	'dd',
	'de',
	'definitely',
	'describe',
	'described',
	'despite',
	'detail',
	'df',
	'di',
	'did',
	'didn',
	'didnt',
	'different',
	'dj',
	'dk',
	'dl',
	'do',
	'does',
	'doesn',
	'doesnt',
	'doing',
	'don',
	'done',
	'dont',
	'down',
	'downwards',
	'dp',
	'dr',
	'ds',
	'dt',
	'du',
	'due',
	'during',
	'dx',
	'dy',
	'e',
	'e2',
	'e3',
	'ea',
	'each',
	'ec',
	'ed',
	'edu',
	'ee',
	'ef',
	'effect',
	'eg',
	'ei',
	'eight',
	'eighty',
	'either',
	'ej',
	'el',
	'eleven',
	'else',
	'elsewhere',
	'em',
	'empty',
	'en',
	'end',
	'ending',
	'enough',
	'entirely',
	'eo',
	'ep',
	'eq',
	'er',
	'es',
	'especially',
	'est',
	'et',
	'et-al',
	'etc',
	'eu',
	'ev',
	'even',
	'ever',
	'every',
	'everybody',
	'everyone',
	'everything',
	'everywhere',
	'ex',
	'exactly',
	'example',
	'except',
	'ey',
	'f',
	'f2',
	'fa',
	'far',
	'fc',
	'few',
	'ff',
	'fi',
	'fifteen',
	'fifth',
	'fify',
	'fill',
	'find',
	'fire',
	'first',
	'five',
	'fix',
	'fj',
	'fl',
	'fn',
	'fo',
	'followed',
	'following',
	'follows',
	'for',
	'former',
	'formerly',
	'forth',
	'forty',
	'found',
	'four',
	'fr',
	'from',
	'front',
	'fs',
	'ft',
	'fu',
	'full',
	'further',
	'furthermore',
	'fy',
	'g',
	'ga',
	'gave',
	'ge',
	'get',
	'gets',
	'getting',
	'gi',
	'give',
	'given',
	'gives',
	'giving',
	'gj',
	'gl',
	'go',
	'goes',
	'going',
	'gone',
	'got',
	'gotten',
	'gr',
	'greetings',
	'gs',
	'gy',
	'h',
	'h2',
	'h3',
	'had',
	'hadn',
	'hadnt',
	'happens',
	'hardly',
	'has',
	'hasn',
	'hasnt',
	'hasnt',
	'have',
	'haven',
	'havent',
	'having',
	'he',
	'hed',
	'hed',
	'hell',
	'hello',
	'help',
	'hence',
	'her',
	'here',
	'hereafter',
	'hereby',
	'herein',
	'heres',
	'heres',
	'hereupon',
	'hers',
	'herself',
	'hes',
	'hes',
	'hh',
	'hi',
	'hid',
	'him',
	'himself',
	'his',
	'hither',
	'hj',
	'ho',
	'home',
	'hopefully',
	'how',
	'howbeit',
	'however',
	'hows',
	'hr',
	'hs',
	'http',
	'hu',
	'hundred',
	'hy',
	'i',
	'i2',
	'i3',
	'i4',
	'i6',
	'i7',
	'i8',
	'ia',
	'ib',
	'ibid',
	'ic',
	'id',
	'id',
	'ie',
	'if',
	'ig',
	'ignored',
	'ih',
	'ii',
	'ij',
	'il',
	'ill',
	'im',
	'im',
	'immediate',
	'immediately',
	'importance',
	'important',
	'in',
	'inasmuch',
	'inc',
	'indeed',
	'index',
	'indicate',
	'indicated',
	'indicates',
	'information',
	'inner',
	'insofar',
	'instead',
	'interest',
	'into',
	'invention',
	'inward',
	'io',
	'ip',
	'iq',
	'ir',
	'is',
	'isn',
	'isnt',
	'it',
	'itd',
	'itd',
	'itll',
	'its',
	'its',
	'itself',
	'iv',
	'ive',
	'ix',
	'iy',
	'iz',
	'j',
	'jj',
	'jr',
	'js',
	'jt',
	'ju',
	'just',
	'k',
	'ke',
	'keep',
	'keeps',
	'kept',
	'kg',
	'kj',
	'km',
	'know',
	'known',
	'knows',
	'ko',
	'l',
	'l2',
	'la',
	'largely',
	'last',
	'lately',
	'later',
	'latter',
	'latterly',
	'lb',
	'lc',
	'le',
	'least',
	'les',
	'less',
	'lest',
	'let',
	'lets',
	'lets',
	'lf',
	'like',
	'liked',
	'likely',
	'line',
	'little',
	'lj',
	'll',
	'll',
	'ln',
	'lo',
	'look',
	'looking',
	'looks',
	'los',
	'lr',
	'ls',
	'lt',
	'ltd',
	'm',
	'm2',
	'ma',
	'made',
	'mainly',
	'make',
	'makes',
	'many',
	'may',
	'maybe',
	'me',
	'mean',
	'means',
	'meantime',
	'meanwhile',
	'merely',
	'mg',
	'might',
	'mightn',
	'mightnt',
	'mill',
	'million',
	'mine',
	'miss',
	'ml',
	'mn',
	'mo',
	'more',
	'moreover',
	'most',
	'mostly',
	'move',
	'mr',
	'mrs',
	'ms',
	'mt',
	'mu',
	'much',
	'mug',
	'must',
	'mustn',
	'mustnt',
	'my',
	'myself',
	'n',
	'n2',
	'na',
	'name',
	'namely',
	'nay',
	'nc',
	'nd',
	'ne',
	'near',
	'nearly',
	'necessarily',
	'necessary',
	'need',
	'needn',
	'neednt',
	'needs',
	'neither',
	'never',
	'nevertheless',
	'new',
	'next',
	'ng',
	'ni',
	'nine',
	'ninety',
	'nj',
	'nl',
	'nn',
	'no',
	'nobody',
	'non',
	'none',
	'nonetheless',
	'noone',
	'nor',
	'normally',
	'nos',
	'not',
	'noted',
	'nothing',
	'novel',
	'now',
	'nowhere',
	'nr',
	'ns',
	'nt',
	'ny',
	'o',
	'oa',
	'ob',
	'obtain',
	'obtained',
	'obviously',
	'oc',
	'od',
	'of',
	'off',
	'often',
	'og',
	'oh',
	'oi',
	'oj',
	'ok',
	'okay',
	'ol',
	'old',
	'om',
	'omitted',
	'on',
	'once',
	'one',
	'ones',
	'only',
	'onto',
	'oo',
	'op',
	'oq',
	'or',
	'ord',
	'os',
	'ot',
	'other',
	'others',
	'otherwise',
	'ou',
	'ought',
	'our',
	'ours',
	'ourselves',
	'out',
	'outside',
	'over',
	'overall',
	'ow',
	'owing',
	'own',
	'ox',
	'oz',
	'p',
	'p1',
	'p2',
	'p3',
	'page',
	'pagecount',
	'pages',
	'par',
	'part',
	'particular',
	'particularly',
	'pas',
	'past',
	'pc',
	'pd',
	'pe',
	'per',
	'perhaps',
	'pf',
	'ph',
	'pi',
	'pj',
	'pk',
	'pl',
	'placed',
	'please',
	'plus',
	'pm',
	'pn',
	'po',
	'poorly',
	'possible',
	'possibly',
	'potentially',
	'pp',
	'pq',
	'pr',
	'predominantly',
	'present',
	'presumably',
	'previously',
	'primarily',
	'probably',
	'promptly',
	'proud',
	'provides',
	'ps',
	'pt',
	'pu',
	'put',
	'py',
	'q',
	'qj',
	'qu',
	'que',
	'quickly',
	'quite',
	'qv',
	'r',
	'r2',
	'ra',
	'ran',
	'rather',
	'rc',
	'rd',
	're',
	'readily',
	'really',
	'reasonably',
	'recent',
	'recently',
	'ref',
	'refs',
	'regarding',
	'regardless',
	'regards',
	'related',
	'relatively',
	'research',
	'research-articl',
	'respectively',
	'resulted',
	'resulting',
	'results',
	'rf',
	'rh',
	'ri',
	'right',
	'rj',
	'rl',
	'rm',
	'rn',
	'ro',
	'rq',
	'rr',
	'rs',
	'rt',
	'ru',
	'run',
	'rv',
	'ry',
	's',
	's2',
	'sa',
	'said',
	'same',
	'saw',
	'say',
	'saying',
	'says',
	'sc',
	'sd',
	'se',
	'sec',
	'second',
	'secondly',
	'section',
	'see',
	'seeing',
	'seem',
	'seemed',
	'seeming',
	'seems',
	'seen',
	'self',
	'selves',
	'sensible',
	'sent',
	'serious',
	'seriously',
	'seven',
	'several',
	'sf',
	'shall',
	'shan',
	'shant',
	'she',
	'shed',
	'shed',
	'shell',
	'shes',
	'shes',
	'should',
	'shouldn',
	'shouldnt',
	'shouldve',
	'show',
	'showed',
	'shown',
	'showns',
	'shows',
	'si',
	'side',
	'significant',
	'significantly',
	'similar',
	'similarly',
	'since',
	'sincere',
	'six',
	'sixty',
	'sj',
	'sl',
	'slightly',
	'sm',
	'sn',
	'so',
	'some',
	'somebody',
	'somehow',
	'someone',
	'somethan',
	'something',
	'sometime',
	'sometimes',
	'somewhat',
	'somewhere',
	'soon',
	'sorry',
	'sp',
	'specifically',
	'specified',
	'specify',
	'specifying',
	'sq',
	'sr',
	'ss',
	'st',
	'still',
	'stop',
	'strongly',
	'sub',
	'substantially',
	'successfully',
	'such',
	'sufficiently',
	'suggest',
	'sup',
	'sure',
	'sy',
	'system',
	'sz',
	't',
	't1',
	't2',
	't3',
	'take',
	'taken',
	'taking',
	'tb',
	'tc',
	'td',
	'te',
	'tell',
	'ten',
	'tends',
	'tf',
	'th',
	'than',
	'thank',
	'thanks',
	'thanx',
	'that',
	'thatll',
	'thats',
	'thats',
	'thatve',
	'the',
	'their',
	'theirs',
	'them',
	'themselves',
	'then',
	'thence',
	'there',
	'thereafter',
	'thereby',
	'thered',
	'therefore',
	'therein',
	'therell',
	'thereof',
	'therere',
	'theres',
	'theres',
	'thereto',
	'thereupon',
	'thereve',
	'these',
	'they',
	'theyd',
	'theyd',
	'theyll',
	'theyre',
	'theyre',
	'theyve',
	'thickv',
	'thin',
	'think',
	'third',
	'this',
	'thorough',
	'thoroughly',
	'those',
	'thou',
	'though',
	'thoughh',
	'thousand',
	'three',
	'throug',
	'through',
	'throughout',
	'thru',
	'thus',
	'ti',
	'til',
	'tip',
	'tj',
	'tl',
	'tm',
	'tn',
	'to',
	'together',
	'too',
	'took',
	'top',
	'toward',
	'towards',
	'tp',
	'tq',
	'tr',
	'tried',
	'tries',
	'truly',
	'try',
	'trying',
	'ts',
	'ts',
	'tt',
	'tv',
	'twelve',
	'twenty',
	'twice',
	'two',
	'tx',
	'u',
	'u201d',
	'ue',
	'ui',
	'uj',
	'uk',
	'um',
	'un',
	'under',
	'unfortunately',
	'unless',
	'unlike',
	'unlikely',
	'until',
	'unto',
	'uo',
	'up',
	'upon',
	'ups',
	'ur',
	'us',
	'use',
	'used',
	'useful',
	'usefully',
	'usefulness',
	'uses',
	'using',
	'usually',
	'ut',
	'v',
	'va',
	'value',
	'various',
	'vd',
	've',
	've',
	'very',
	'via',
	'viz',
	'vj',
	'vo',
	'vol',
	'vols',
	'volumtype',
	'vq',
	'vs',
	'vt',
	'vu',
	'w',
	'wa',
	'want',
	'wants',
	'was',
	'wasn',
	'wasnt',
	'wasnt',
	'way',
	'we',
	'wed',
	'wed',
	'welcome',
	'well',
	'well',
	'well-b',
	'went',
	'were',
	'were',
	'weren',
	'werent',
	'werent',
	'weve',
	'what',
	'whatever',
	'whatll',
	'whats',
	'whats',
	'when',
	'whence',
	'whenever',
	'whens',
	'where',
	'whereafter',
	'whereas',
	'whereby',
	'wherein',
	'wheres',
	'wheres',
	'whereupon',
	'wherever',
	'whether',
	'which',
	'while',
	'whim',
	'whither',
	'who',
	'whod',
	'whoever',
	'whole',
	'wholl',
	'whom',
	'whomever',
	'whos',
	'whos',
	'whose',
	'why',
	'whys',
	'wi',
	'widely',
	'will',
	'willing',
	'wish',
	'with',
	'within',
	'without',
	'wo',
	'won',
	'wonder',
	'wont',
	'wont',
	'words',
	'world',
	'would',
	'wouldn',
	'wouldnt',
	'wouldnt',
	'www',
	'x',
	'x1',
	'x2',
	'x3',
	'xf',
	'xi',
	'xj',
	'xk',
	'xl',
	'xn',
	'xo',
	'xs',
	'xt',
	'xv',
	'xx',
	'y',
	'y2',
	'yes',
	'yet',
	'yj',
	'yl',
	'you',
	'youd',
	'youd',
	'youll',
	'your',
	'youre',
	'youre',
	'yours',
	'yourself',
	'yourselves',
	'youve',
	'yr',
	'ys',
	'yt',
	'z',
	'zero',
	'zi',
	'zz',
}

In [136]:
def normalize(text: str) -> str:
    """
    Normalize text from a pandas entry to ASCII-only lowercase characters. Hence, this removes Unicode characters with no ASCII
    equivalent (e.g., emojis and CJKs).

    Do not use this function alone, use `clean_and_tokenize()`.
    
    # Parameters
    * text: String entry.

    # Returns
    ASCII-normalized text containing only lowercase letters.

    # Examples
    normalize("¿Cómo estás?")
    $ 'como estas?'

    normalize(" hahahaha HUY! Kamusta 😅 Mayaman $$$ ka na ba?") 
    $ ' hahahaha huy! kamusta  mayaman $$$ ka na ba?'
    """
    normalized = unicodedata.normalize('NFKD', text)
    ascii_text = normalized.encode('ascii', 'ignore').decode('ascii')

    return ascii_text.lower()


def rem_punctuation(text: str) -> str:
    """
    Removes the punctuations. This function simply replaces all punctuation marks and special characters
    to the empty string. Hence, for symbols enclosed by whitespace, the whitespace are not collapsed to a single whitespace
    (for more information, see the examples).

    Do not use this function alone, use `clean_and_tokenize()`.
    
    # Parameters
    * text: String entry.

    # Returns
    Text with the punctuation removed, or None if the result is empty or input invalid.

    # Examples
    rem_punctuation("this word $$ has two spaces after it!")
    $ 'this word  has two spaces after it'

    rem_punctuation("these!words@have$no%space")
    $ 'thesewordshavenospace'
    """
    return re.sub(f"[{re.escape(string.punctuation)}]", "", text)


def collapse_whitespace(text: str) -> str:
    """
    This collapses whitespace. Here, collapsing means the transduction of all whitespace strings of any
    length to a whitespace string of unit length (e.g., "   " -> " "; formally " "+ -> " ").
    
    Do not use this function alone, use `clean_and_tokenize()`.

    # Parameters
    * text: String entry.

    # Returns
    Text with the whitespaces collapsed.

    # Examples
    collapse_whitespace("  huh,  was.  that!!! ")
    $ 'huh, was. that!!!'
    """
    return re.sub(f" +", " ", text).strip()


def rem_stopwords(tokens: str, stopwords: set[str]) -> str:
    """
    This removes all stopwords. For fast look up, `stopwords` is a set with type str.
    This allows for constant time lookups as opposed to a linear search with a list.

    Strings detected as stopwords is replaced with the empty string "".

    Do not use this function alone, use `clean_and_tokenize()`.

    # Parameters
    * text: A tokenized string. 
    * stopwords: stopword dictionary.
    
    # Returns
    Text with the stopwords removed.

    # Examples
    rem_stopwords(["he", "is", "an", "amazing", "master",], stopwords_lut)
    $ ['amazing', 'master']

    # Future
    If this function is too slow, we may implement `stopwords` as an 26-ary trie to achieve log-linear time.
    """
    filtered = [word for word in tokens if word not in stopwords]
    return filtered
    
def clean_and_tokenize(text: str, stopwords: set[str]) -> list[str]:
    """
    This is the main function for data cleaning (i.e., it calls all the cleaning functions in the prescribed order).

    This function should be used as a first-class function in a map.

    # Parameters
    * text: The string entry from a DataFrame column.
    * stopwords: stopword dictionary.

    # Returns
    Clean tokenized string. 
    """
    # cleaning on the base string
    text = normalize(text)
    text = rem_punctuation(text)
    text = collapse_whitespace(text)
    
    # tokenization is now trivial since the cleaning steps allow the tokenization to be a mere word boundary split
    toks = text.split()

    # cleaning on the tokenized string
    toks = rem_stopwords(toks, stopwords)

    return toks

# perform cleaning stage
df["clean_text"] = df['clean_text'].fillna("")
df["clean_ours"] = df['clean_text'].map(lambda x: clean_and_tokenize(x, stopwords_set))

# sanity check
df.to_csv("Cleaned_Data.csv", index=False)
df.sample(20)

# 3 preprocessing

> 🏗️ Perhaps swap S3 and S4. Refer to literature on what comes first.

This section discusses preprocessing steps for the cleaned data.

---
**References**
1. ()

In [20]:
pass

# 4 exploratory data analysis

This section discusses the exploratory data analysis conducted on the dataset after cleaning.

> Notes from Zhean: <br>
> From manual checking via OpenRefine, there are a total of 162972. `df.info()` should have the same result post-processing.
> Furthermore, there should be two columns, `clean_text` (which is a bit of a misnormer since it is still dirty) contains the Tweets (text data). The second column is the `category` which contains the sentiment of the Tweet and is a tribool (1 positive, 0 neutral or indeterminate, and -1 for negative).

---
**References**
1. ()